# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846645099175                   -0.70    4.2         
  2   -7.852279049319       -2.25       -1.53    1.0   22.6ms
  3   -7.852607827909       -3.48       -2.56    1.8   25.2ms
  4   -7.852645926318       -4.42       -2.85    2.5   31.4ms
  5   -7.852646420866       -6.31       -3.07    1.2   24.1ms
  6   -7.852646679018       -6.59       -4.07    1.0   22.9ms
  7   -7.852646686580       -8.12       -4.74    2.0   76.6ms
  8   -7.852646686715       -9.87       -5.44    1.5   25.3ms
  9   -7.852646686727      -10.92       -5.62    1.8   26.6ms
 10   -7.852646686730      -11.64       -6.78    1.0   23.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846800570245                   -0.70           4.8         
  2   -7.852526233611       -2.24       -1.64   0.80    2.2    235ms
  3   -7.852635639405       -3.96       -2.73   0.80    1.0   21.1ms
  4   -7.852646476103       -4.97       -3.24   0.80    2.2   27.2ms
  5   -7.852646668960       -6.71       -4.06   0.80    1.0   20.6ms
  6   -7.852646686374       -7.76       -4.71   0.80    2.0   26.2ms
  7   -7.852646686719       -9.46       -5.45   0.80    2.0   25.8ms
  8   -7.852646686730      -10.98       -6.79   0.80    1.8   24.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.332785e+01     3.706568e+00
 * time: 0.4216299057006836
     1     1.084173e+00     2.009595e+00
 * time: 0.6115908622741699
     2    -1.342657e+00     2.382133e+00
 * time: 0.6339888572692871
     3    -3.557018e+00     2.111651e+00
 * time: 0.6662600040435791
     4    -4.527987e+00     1.927488e+00
 * time: 0.6982748508453369
     5    -6.423306e+00     1.261537e+00
 * time: 0.7303268909454346
     6    -7.253818e+00     7.109531e-01
 * time: 0.7622017860412598
     7    -7.534074e+00     6.888555e-01
 * time: 0.7844038009643555
     8    -7.708005e+00     2.362437e-01
 * time: 0.8066380023956299
     9    -7.762491e+00     8.574024e-02
 * time: 0.8289239406585693
    10    -7.795191e+00     8.324705e-02
 * time: 0.8514559268951416
    11    -7.820300e+00     8.418323e-02
 * time: 0.8739728927612305
    12    -7.836263e+00     5.561215e-02
 * time: 0.8967728614807129
    13    -7.846794e+00     3.709933e-02
 * time: 0.9190878868

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846791562286                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645804916                   -1.64         
  2   -7.852646686730       -6.05       -3.71    1.57s
  3   -7.852646686730      -13.06       -7.16    157ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.494233181759162e-7
|ρ_newton - ρ_scfv| = 1.8563207684647625e-7
|ρ_newton - ρ_dm|   = 2.9436017260596223e-9
